# TREND FOLLOWING
In this experiment, I would try to derive a exit method/condition for trend-following strategies. This inherently involves identifying and labelling trends

### Proposed Methods
- Two Moving Average Method
- ATR/Moving Average Method

### Deprecated Methods
- Directly classifying/clustering windows of returns. 
  - Reason for deprecation : Wasn't robust to noise
- Using permutation entropy (or any other entropies) in addition to clustering
  - Reason for deprecation : Couldn't model the problem appropriately
- Using tr8dr's AmplitudeLabeller
  - Reason for deprecation : Couldn't label out-of-sample data


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf

In [2]:
full_daterange = pd.date_range('2000-01-01', '2024-04-20', freq='D')
train_daterange = pd.date_range('2000-01-01', '2019-12-31', freq='D')
test_daterange = pd.date_range('2020-01-01', '2024-04-30', freq='D')

def clean_data(_data : pd.DataFrame, keep_volume=False, compute_target=False, look_forward=15, upper_factor=3, lower_factor=3, **kwargs):
    data = _data.copy()
    data.columns = data.columns.str.lower()

    columns = ['atr', 'returns']
    passed_columns = kwargs.get('columns', None)

    if passed_columns:
        if not isinstance(passed_columns, list):
            passed_columns = list(passed_columns)

        columns = passed_columns + columns

    data['atr'] = ta.atr(data['high'], data['low'], data['close'], length=5)
    # rolling_mean = data['_atr'].rolling(window=5).mean()
    # rolling_std = data['_atr'].rolling(window=5).std()
    # data.loc[:, 'atr'] = (data['_atr'] - rolling_mean) / rolling_std

    data.loc[:, 'returns'] = data['close'].pct_change(fill_method=None).fillna(0)
    # rolling_mean_return = data['_returns'].rolling(window=5).mean()
    # rolling_std_return = data['_returns'].rolling(window=5).std()
    # data.loc[:, 'returns'] = (data['_returns'] - rolling_mean_return) / rolling_std_return

    if keep_volume:
        data.loc[:, 'volume_change'] = data['volume'].pct_change(fill_method=None)
        columns.append('volume_change')

    data = data.dropna()

    if compute_target:
        columns.append('target')
        columns.append('target_period')
        labels = []
        bar_counts = []  # List to store the count of bars until a barrier is touched

        for i in range(len(data)):
            price = data['close'].iloc[i]
            _atr = data['atr'].iloc[i]
            upper_barrier = price + (_atr * upper_factor)
            lower_barrier = price - (_atr * lower_factor)

            forward_prices = data['close'].iloc[i+1:i+1+look_forward]

            upper_cross = forward_prices[forward_prices >= upper_barrier]
            lower_cross = forward_prices[forward_prices <= lower_barrier]

            if not upper_cross.empty and not lower_cross.empty:
                first_upper_index = data.index.get_loc(upper_cross.index[0]) - i
                first_lower_index = data.index.get_loc(lower_cross.index[0]) - i
                if first_upper_index < first_lower_index:
                    label = 1
                    bars_away = first_upper_index
                else:
                    label = -1
                    bars_away = first_lower_index
            elif not upper_cross.empty:
                label = 1
                bars_away = data.index.get_loc(upper_cross.index[0]) - i
            elif not lower_cross.empty:
                label = -1
                bars_away = data.index.get_loc(lower_cross.index[0]) - i
            else:
                label = 0  # No barrier touched
                bars_away = look_forward  # Indicates no touch within the look forward window

            labels.append(label)
            bar_counts.append(bars_away)

        data['target'] = labels
        data['target_period'] = bar_counts  # Adding the new column to the DataFrame

    data = data.dropna()
    data = data[data.index.isin(full_daterange)]
        

    return data[columns]

In [3]:
# DOWNLOAD YAHOO FINANCE DATA

# Define ticker symbols
es_ticker = 'EURUSD=X' # "ES=F"  # S&P 500 as a proxy for ES futures
dxy_ticker = "DX-Y.NYB"
vix_ticker = "^VIX"

df_es = yf.download(es_ticker)
df_dxy = yf.download(dxy_ticker)
df_vix = yf.download(vix_ticker)

clean_df_es = clean_data(df_es, keep_volume=False, compute_target=True, columns=['open', 'high', 'low', 'close'])
clean_df_vix = clean_data(df_vix)
clean_df_dxy = clean_data(df_dxy)

full_df = clean_df_es.join([clean_df_dxy.add_prefix('dxy_'), clean_df_vix.add_prefix('vix_')], how='inner')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [35]:
# Functions to Visualize and Evaluate Labels
window_size = 5 # One week
n_components = 5 # Number of states


def train_test_split(data:pd.DataFrame=None):
    if data is None:
        data = full_df.copy()

    train_df = data[data.index.isin(train_daterange)]
    test_df = data[data.index.isin(test_daterange)]

    return train_df, test_df


def visualize_clustering(labels, daterange=None, adds=None):

    df = full_df.copy()

    if daterange is not None:
        df = df[df.index.isin(daterange)]

    df['labels'] = labels
    df['labels'] = df['labels'].astype(str)

    # Using Plotly Express to create a scatter plot
    fig = px.scatter(df, x=df.index, y='close',
                    color='labels',  # This will use a distinct color for each label
                    labels={'x': 'Date', 'Price': 'Price'},  # Custom labels for axes
                    category_orders={'labels': sorted(df['labels'].unique())},  # Optional: Order of categories
                    title='Price Scatter Plot Colored by Labels',
                    color_discrete_sequence=px.colors.qualitative.Plotly)  # Using a qualitative color scale
    
    if adds is not None:
        fig.add_scatter(x=df.index, y=adds, mode='markers', name='Additional Data')

    # Show the plot
    fig.show()

    return df['labels']


def evaluate_clustering(labels, labels_long:list, labels_short:list, shift=False, daterange=None):

    df = full_df.copy()

    if daterange is not None:
        df = df[df.index.isin(daterange)]

    df['labels'] = labels

    signals = np.where(np.isin(df['labels'], labels_long), 1,
                       np.where(np.isin(df['labels'], labels_short), -1, 0))
    
    rets = signals * df['returns'] if not shift else signals * df['returns'].shift(-1)

    fig = go.Figure(data=go.Scatter(x=df.index, y=np.cumsum(rets)))
    fig.update_layout(title='Cumulative Returns',
                      xaxis_title='Time',
                      yaxis_title='Cumulative Returns')
    fig.show()


## BENCHMARK : TWO MOVING AVERAGES

In [5]:
benchmark_df = full_df.copy()

fast_ma = ta.ema(benchmark_df['close'], 5)
slow_ma = ta.ema(benchmark_df['close'], 22)

benchmark_df['labels'] = np.where(
    (benchmark_df['close'] > fast_ma) & (fast_ma > slow_ma), 1, # Strong Bullish
    np.where(
        (benchmark_df['close'] > fast_ma) & (slow_ma >= fast_ma), 0, # Weak Bullish, 
        np.where(
            (benchmark_df['close'] < fast_ma) & (fast_ma < slow_ma), -1, # Strong Bearish
            np.where(
                (benchmark_df['close'] < fast_ma) & (slow_ma <= fast_ma), 0, # Weak Bullish,
                0
            )
        )
    ) 
)

In [6]:
labels_benchmark = visualize_clustering(benchmark_df['labels'])

In [7]:
evaluate_clustering(benchmark_df['labels'], [1], [-1])

## METHOD : ATR-MOVING AVERAGE

,open,high,low,close,atr,returns,target,target_period,dxy_atr,dxy_returns,vix_atr,vix_returns,labels_5,labels
Date,,,,,,,,,,,,,,
2003-12-08,1.216797,1.224005,1.215407,1.222001,0.010232,0.002713,0,15,0.691857,-0.004375,0.940354,-0.032183,0,0
2003-12-09,1.222105,1.227702,1.219795,1.224995,0.009602,0.002450,1,15,0.671485,-0.001577,1.004283,0.065901,0,0
2003-12-10,1.224905,1.226603,1.216205,1.219096,0.009803,-0.004815,1,14,0.679188,0.002708,0.941426,0.013613,0,0
2003-12-11,1.219096,1.223496,1.212298,1.222404,0.010139,0.002714,1,13,0.699350,0.000338,1.041141,-0.063794,0,0
2003-12-12,1.222703,1.230603,1.221300,1.227898,0.009946,0.004494,1,13,0.667480,-0.004388,1.010913,-0.019127,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-15,1.064963,1.066553,1.062361,1.064963,0.009953,-0.007199,0,15,0.591932,0.001603,2.469523,0.110919,0,0
2024-04-16,1.062575,1.065405,1.060412,1.062575,0.008961,-0.002242,0,15,0.563545,0.000471,2.359619,-0.043162,0,0
2024-04-17,1.062146,1.065269,1.060817,1.062146,0.008060,-0.000404,0,15,0.564836,-0.002917,2.201695,-0.010326,0,0


In [62]:
labeler_df = full_df.copy()

threshold_amplitude = 0.5 # multiples of ATR
inactive_period = 10 # number of bars without significant moves
ema_length = 22 # ema length

labeler_atr = ta.atr(labeler_df['high'], labeler_df['low'], labeler_df['close'], 14)
labeler_ema = ta.ema(labeler_df['close'], ema_length)

labeler_minamp = labeler_atr * threshold_amplitude
labeler_bodysize = np.absolute(labeler_df['open'] - labeler_df['close'])
labeler_strong_bar = labeler_bodysize >= labeler_minamp
labeler_strong_bull = labeler_strong_bar & (labeler_df['open'] < labeler_df['close'])
labeler_strong_bear = labeler_strong_bar & (labeler_df['open'] > labeler_df['close'])

labeler_regime = []
labeler_inactive_count = []

regime = 0
inactive_count = 0

for i in range(len(labeler_df)):
    _close = labeler_df['close'].iloc[i]
    _ema = labeler_ema.iloc[i]
    _strong_bull = labeler_strong_bull.iloc[i]
    _strong_bear = labeler_strong_bear.iloc[i]
    
    # For Neutral Regime
    if regime == 0:
        if _strong_bull and (_close > _ema):
            regime = 2
            inactive_count = 0
    
        elif _strong_bear and (_close < _ema):
            regime = -2
            inactive_count = 0

    # For Bullish Regime
    elif regime > 0:
        if _strong_bull:
            regime = +2 # Reinforce Regime
            inactive_count = 0 # Reset count with every significant move

        else:
            # Count insignificant moves
            inactive_count += 1
            
            # Check for maximum inactive period
            if inactive_count >= inactive_period:
                regime = 0

            elif _strong_bear:
                # Regime switched to Bearish
                if _close < _ema:
                    regime = -2

                # Weak Bullish Regime
                else:
                    regime = 1
            

    # For Bearish Regime
    elif regime < 0:
        if _strong_bear:
            regime = -2 # Reinforce Regime
            inactive_count = 0 # Reset count with every significant move
        
        else:
            # Count insignificant moves
            inactive_count += 1

            # Check for maximum inactive period
            if inactive_count >= inactive_period:
                regime = 0

            # Reg ime switched to Bearish
            elif _strong_bull:
                if _close > _ema:
                    regime = 2

                # Weak Bearish Regime
                else:
                    regime =-1
        

    labeler_regime.append(regime)
    labeler_inactive_count.append(inactive_count)

labeler_df['labels_5'] = labeler_regime
labeler_df['labels'] = np.where(labeler_df['labels_5'] > 0, 1,
                                np.where(labeler_df['labels_5'] < 0, -1,
                                         0))

In [61]:
labels_atrma = visualize_clustering(labeler_df['labels'], adds=labeler_ema)

In [10]:
evaluate_clustering(labeler_df['labels'], [1], [-1])

## EXPERIMENT : FIT A HIDDEN MARKOV MODEL TO THE LABELS
Use this to find the probabilities of a regime change in a lookahead period.

Steps:
- TRAIN THE MODEL
  - Prepare the data : We would use the labels from Amplitude Labeller
  - Build the model : We would build a 3-state Markov Model
  - Estimate the parameters for the HMM Model
    - Transitions Matrix
    - Emission Matrix




In [11]:
from hmmlearn import hmm


def hmm_model(data, n_components=3, multinomial=False):
    daterange = data.index
    data = np.array(data)[:, np.newaxis]

    if multinomial:
        data[data == -1] = 2
        model = hmm.MultinomialHMM(n_components=n_components, n_iter=100)
    else:
        model = hmm.CategoricalHMM(n_components=n_components, n_iter=100)

    # Fit Model to Data
    model.fit(data)

    # Predict the hidden states for your observations
    hidden_states = model.predict(data)

    
    visualize_clustering(hidden_states, daterange=daterange)

    return model, hidden_states

def state_probability_after_n_steps(initial_state, n_steps, trans_matrix):
    # Convert initial state index to a one-hot encoded vector
    num_states = trans_matrix.shape[0]
    initial_prob = np.zeros(num_states)
    initial_prob[initial_state] = 1
    
    # Matrix exponentiation
    prob_matrix = np.linalg.matrix_power(trans_matrix, n_steps)
    
    # Multiply initial state probability vector by the probability matrix
    future_prob = np.dot(initial_prob, prob_matrix)
    
    return future_prob

def state_probability_given_history(model, history, n_steps):
    """
    Calculate the probability distribution of states after n steps given a history of observed states.
    
    Parameters:
        model (CategoricalHMM): The trained HMM model.
        history (list or array): The sequence of observed states leading up to the current time.
        n_steps (int): The number of steps to project the state probabilities into the future.

    Returns:
        np.array: A probability distribution over states after n steps.
    """
    # Convert history to appropriate shape
    history = np.array(history).reshape(-1, 1)
    
    # Compute the log probabilities and responsibilities
    logprob, fwdlattice = model.score_samples(history)
    
    # Extract the last row of forward probabilities and normalize it
    current_distribution = np.exp(fwdlattice[-1])
    current_distribution /= np.sum(current_distribution)
    
    # Compute the state distribution after n steps using matrix exponentiation
    trans_matrix_n_steps = np.linalg.matrix_power(model.transmat_, n_steps)
    
    # Calculate the future state distribution
    future_distribution = np.dot(current_distribution, trans_matrix_n_steps)
    
    return future_distribution


In [12]:
train_data, test_data = train_test_split(labeler_df)
train_data.shape, test_data.shape

((4013, 14), (1082, 14))

In [13]:
np.random.seed(14)

model, hmm_labels = hmm_model(train_data['labels'].astype(int).replace(-1, 2))
transition_matrix = model.transmat_
evaluate_clustering(hmm_labels, [2], [0], shift=True, daterange=train_data.index)

## EXPERIMENT : PREDICT NEXT LABELS USING ML
Use this to find the probabilities of a regime change in a lookahead period.

Steps:
- TRAIN THE MODEL
  - Prepare the data : We would use the labels from Amplitude Labeller
  - Build the model : We would build a 3-state Markov Model
  - Estimate the parameters for the HMM Model
    - Transitions Matrix
    - Emission Matrix

In [14]:
import xgboost as xgb
from keras.layers import LSTM, Dense, Dropout, InputLayer  # type: ignore
from keras.models import Sequential  # type: ignore
from keras.regularizers import l2  # type: ignore
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Assuming predictions_rounded and y_test are your predicted and true labels

# # Classification report
# report = classification_report(y_test, predictions_rounded)
# print(report)

# # Individual scores
# precision = precision_score(y_test, predictions_rounded, average='weighted')
# recall = recall_score(y_test, predictions_rounded, average='weighted')
# f1 = f1_score(y_test, predictions_rounded, average='weighted')

# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')



In [15]:
label_data_X = labeler_df[['returns', 'dxy_atr', 'dxy_returns', 'vix_atr', 'vix_returns']].copy()
label_data_Y = labeler_df['labels'].replace(-1, 2).copy()

In [16]:

# Assuming df is your dataframe and labels is your vector of labels
# Example: df = pd.DataFrame(data) and labels = pd.Series(labels)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(label_data_X, label_data_Y, test_size=0.3, random_state=14)

# Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predicting the next value
predictions = model.predict(X_test)
predictions_rounded = np.round(predictions).astype(int)

# Evaluating the model
report = classification_report(y_test, predictions_rounded, zero_division=0)
print(report)

# # Getting feature importance
# feature_importance = np.abs(model.coef_[0])
# feature_names = label_data_X.columns  # replace df with your actual dataframe variable name
# feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
# feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# print(feature_importance_df)


              precision    recall  f1-score   support

           0       0.69      0.99      0.81      1052
           1       0.00      0.00      0.00       250
           2       0.17      0.01      0.02       227

    accuracy                           0.69      1529
   macro avg       0.29      0.33      0.28      1529
weighted avg       0.50      0.69      0.56      1529



In [17]:
# XGBOOST CLASSIFIER

# XGBoost Model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Predicting the next value
predictions = model.predict(X_test)
predictions_rounded = np.round(predictions).astype(int)

# Evaluating the model
report = classification_report(y_test, predictions_rounded, zero_division=0)
print(report)


              precision    recall  f1-score   support

           0       0.80      0.92      0.86      1052
           1       0.50      0.27      0.35       250
           2       0.47      0.37      0.41       227

    accuracy                           0.73      1529
   macro avg       0.59      0.52      0.54      1529
weighted avg       0.70      0.73      0.71      1529



In [18]:
# XGBOOST REGRESSOR

# Assuming label_data_X is your dataframe and label_data_Y is your vector of labels
# Example: df = pd.DataFrame(data) and labels = pd.Series(labels)

# Encoding labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(label_data_Y)

# Scaling features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(label_data_X)

# Creating windowed data function
def create_windowed_data(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size)])
        y.append(labels[i + window_size])
    return np.array(X), np.array(y)

# Creating windowed data
window_size = 5
X, y = create_windowed_data(df_scaled, labels_encoded, window_size)

# Reshaping data for XGBoost
X = X.reshape((X.shape[0], window_size * df_scaled.shape[1]))

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# XGBoost Regressor Model
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Predicting the next value
predictions = model.predict(X_test)
predictions_rounded = np.round(predictions).astype(int)

# Evaluating the model
report = classification_report(y_test, predictions_rounded, zero_division=0)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1055
           1       0.38      0.73      0.50       222
           2       0.71      0.36      0.47       250

    accuracy                           0.78      1527
   macro avg       0.68      0.66      0.63      1527
weighted avg       0.83      0.78      0.79      1527



In [19]:
# LSTM WITH INDEPENDENT FEATURES


# Assuming label_data_X is your dataframe and label_data_Y is your vector of labels
# Example: df = pd.DataFrame(data) and labels = pd.Series(labels)

# Encoding labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(label_data_Y)

# Scaling features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(label_data_X)

# Reshaping data for LSTM
X = np.array(df_scaled).reshape((df_scaled.shape[0], 1, df_scaled.shape[1]))
y = np.array(labels_encoded)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# LSTM Model with regularization
model = Sequential()
model.add(InputLayer(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax', kernel_regularizer=l2(0.01)))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.6812 - loss: 0.8297
Accuracy: 0.6867233514785767


In [20]:
# WINDOWED LSTM

# Function to create windowed data
def create_windowed_data(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size)])
        y.append(labels[i + window_size])
    return np.array(X), np.array(y)

# Encoding labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(label_data_Y)

# Scaling features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(label_data_X)

# Creating windowed data
window_size = 5
X, y = create_windowed_data(df_scaled, labels_encoded, window_size)

# Reshaping data for LSTM
X = X.reshape((X.shape[0], window_size, df_scaled.shape[1]))

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# LSTM Model with regularization
model = Sequential()
model.add(InputLayer(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(labels_encoded)), activation='softmax', kernel_regularizer=l2(0.01)))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.8611 - loss: 0.3893
Accuracy: 0.8677144646644592
